In [3]:
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [6]:
data=pd.read_csv("Bank_Personal_Loan_Modelling.csv")
y=data["Personal Loan"]
data["Loan"]=y
data.drop(columns="Personal Loan",inplace=True)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Loan
0,1,25,1,49,91107,4,1.6,1,0,1,0,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,1,0,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,1,0,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,1,0,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,1,0,0


In [13]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
X.shape

(5000, 13)

In [8]:

enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()
Y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.5, random_state=2)
n_features = X.shape[1]
n_classes = Y.shape[1]

In [14]:
model = Sequential()
model.add(Dense(32, input_dim=13, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [15]:

model.summary()

score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 32)                448       
                                                                 
 dense_3 (Dense)             (None, 2)                 66        
                                                                 
Total params: 514
Trainable params: 514
Non-trainable params: 0
_________________________________________________________________
79/79 [==============================] - 1s 2ms/step - loss: 0.5631 - accuracy: 0.7748
Test loss: 0.5631136298179626
Test accuracy: 0.7748000025749207


In [17]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape)+index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights


shape=get_shape(model)

def fitness_function(weights):
    model.set_weights(set_shape(weights,shape))
    return model.evaluate(X_test, Y_test)[1]



def ant_colony_optimization(fitness_function, n_ants, n_iterations, pheromone_factor, evaporation_rate):
    n_weights = sum(len(w.flatten()) for w in model.get_weights())
    pheromone_trail = np.ones(n_weights)

    best_solution = None
    best_fitness = -1
    
    for iteration in range(n_iterations):
        solutions = []
        for ant in range(n_ants):
     
            solution = np.random.normal(size=n_weights)
            solution *= pheromone_trail
            fitness = fitness_function(solution)
        
            if fitness > best_fitness:
                best_solution = solution
                best_fitness = fitness
                
            solutions.append((solution, fitness))
        

        for weight in range(n_weights):
            pheromone_update = 0
            for solution, fitness in solutions:
                pheromone_update += solution[weight] * (fitness - best_fitness)
            pheromone_trail[weight] = pheromone_trail[weight] * evaporation_rate + pheromone_factor * pheromone_update
      
   
        print("Iteration {}: Best fitness = {}".format(iteration+1, best_fitness))

    return best_solution



best_weights = ant_colony_optimization(fitness_function, n_ants=10, n_iterations=10, pheromone_factor=0.1, evaporation_rate=0.5)
model.set_weights(set_shape(best_weights,shape))


79/79 [==============================] - 0s 2ms/step - loss: 8.0153 - accuracy: 0.4712
Iteration 1: Best fitness = 0.8460000157356262
79/79 [==============================] - 0s 2ms/step - loss: 3.9221 - accuracy: 0.2532
Iteration 2: Best fitness = 0.9083999991416931
79/79 [==============================] - 0s 2ms/step - loss: 1.2381 - accuracy: 0.3912
Iteration 3: Best fitness = 0.909600019454956
79/79 [==============================] - 0s 2ms/step - loss: 0.6238 - accuracy: 0.7524
Iteration 4: Best fitness = 0.909600019454956
79/79 [==============================] - 0s 2ms/step - loss: 0.6860 - accuracy: 0.5760
Iteration 5: Best fitness = 0.9100000262260437
79/79 [==============================] - 0s 2ms/step - loss: 0.7656 - accuracy: 0.0904
Iteration 6: Best fitness = 0.9100000262260437
79/79 [==============================] - 0s 2ms/step - loss: 0.7069 - accuracy: 0.0908
Iteration 7: Best fitness = 0.9100000262260437
79/79 [==============================] - 0s 2ms/step - loss: 0.7

In [18]:

score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

79/79 [==============================] - 0s 2ms/step - loss: 0.6094 - accuracy: 0.9100
Test loss: 0.609375
Test accuracy: 0.9100000262260437
